In [1]:
from __future__ import division
import pyspark as ps    # import the spark suite
import warnings         # display warning if spark context already exists
import os
import pandas as pd
import numpy as np

from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType, FloatType
from pyspark.sql.functions import mean as sql_mean

import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from wordcloud import WordCloud, STOPWORDS
from PIL import Image
import colorsys
import random

import matplotlib.pyplot as plt
% matplotlib inline

In [2]:
try:
    sc = ps.SparkContext('local[4]') # create spark context to work locally on all available cpus
    print('created SparkContext')
except ValueError:
    warnings.warn('SparkContext already exists')    # issue a warning if context already exists

spark = ps.SQLContext(sc)
print('created SQLContext')

created SQLContext


/Users/ophidian/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: UserWarning: SparkContext already exists


In [3]:
# Only Run This Once - It gets angry the second time

data_file = 'data/data.json'
df = spark.read.json(data_file)

df.printSchema()
print "row count: ", df.count()
df.show(3)

root
 |-- author: string (nullable = true)
 |-- excerpt: string (nullable = true)
 |-- title: string (nullable = true)

row count:  9050
+--------------+--------------------+---------------+
|        author|             excerpt|          title|
+--------------+--------------------+---------------+
|CharlesDickens|A CHRISTMAS CAROL...|AChristmasCarol|
|CharlesDickens|Mind! I don't mea...|AChristmasCarol|
|CharlesDickens|Scrooge never pai...|AChristmasCarol|
+--------------+--------------------+---------------+
only showing top 3 rows

